In [9]:
# IMPORTS

# Base
import json
import pandas as pd

# Inner
from scripts.file_manager import YAML_read, JSON_load, JSON_open
from scripts.requests import Request_post

# Old Inner Libs
from scripts.googlesheets_upload_lib import ListConventer
from scripts.intervals_api import *


In [10]:
# Get settings
file = YAML_read('config')
config = file.get_config()
token = config['service_token'] # Exclude API token
link = config['service_link'] # Exclude API url
intervals_credentials = (config['intervals_email'], config['intervals_password'])
json_o = JSON_open('data')
start_data = json_o.open()
actual_data_id = start_data[0]['id'] # This is maximum data id in offline dataset

In [11]:
Request_post(link, token) 

In [12]:
# Get data on cycle request 

result = Request_post(link, token) # First request
data = []
snap = result.result
num = 1
work = True
while work:
    if snap['next']:
        link = snap['next']
        for result in snap['results']:
            if result['id'] > actual_data_id:
                data += [result]
                result = Request_post(link, token)
                snap = result.result
                num += 1
            else:
                print('Actual data not found')
                work = False
                break
        
    else:
        work = False


Actual data not found


In [13]:
# Save data
data = data+start_data
json_l = JSON_load('data')
json_l = json_l.load(data)

In [14]:
# Pandas
df=pd.DataFrame(data)
ids = list(df['identity'].unique())
users_count = len(df['identity'].unique())
print(f'Количество пользователей - {users_count}')
df['date'] = df.apply(lambda x: x['time_create'].split('T')[0],axis=1)
df.head()

Количество пользователей - 214


,id,identity,command,time_create,date
0,2204,2045712808,new_words,2022-12-22T19:40:30.930689+03:00,2022-12-22
1,2203,2045712808,main_menu,2022-12-22T19:40:05.469298+03:00,2022-12-22
2,2202,2045712808,proceed_start,2022-12-22T19:39:50.753952+03:00,2022-12-22
3,2201,2045712808,/start,2022-12-22T19:39:33.652020+03:00,2022-12-22
4,2200,315854463,/menu,2022-12-22T19:11:05.311142+03:00,2022-12-22


In [15]:
# Create intervals token and links wrapper
gic = GetIntervalsConfig()
gic.get_token(intervals_credentials[0], intervals_credentials[1])

Данные АПИ получены


In [16]:
def get_days(row):
    try:
        return len(list(df[df['identity'] == row['identity']]['date'].unique()))
    except:
        0
        
def get_counts(row):
    return df['identity'].value_counts ()[row['identity']]

# Data days, counts
days_stat = pd.DataFrame(ids, columns=['identity'])
days_stat['days'] = days_stat.apply(get_days, axis=1)
days_stat['counts'] = days_stat.apply(get_counts, axis=1)
days_stat = days_stat.sort_values(by='days', ascending=False).reset_index()[['identity', 'days', 'counts']]

# Upload data on intervals
lc = ListConventer(days_stat,'width_headers')
lc.conventer()
IntervalsUpload(gic.token, intervals_link(168), lc.datsheets)   

Данные успешно загружены в контейнер
<Response [200]>


In [17]:
# Data commands
commands = df['command'].value_counts().reset_index()
commands.columns = ['command', 'counts']


# Upload data on intervals
lc = ListConventer(commands,'width_headers')
lc.conventer()
IntervalsUpload(gic.token, intervals_link(169), lc.datsheets)   

Данные успешно загружены в контейнер
<Response [200]>


In [54]:
# Fint extreme id's with so large, or so small values
max_count = days_stat['counts'].quantile(0.9)
min_count = days_stat['counts'].quantile(0.2)
extreme_values = list(days_stat[(days_stat['counts'] < min_count) \
                                | (days_stat['counts'] > max_count)]['identity'])

# Clean dataset
clean_df = df[~df['identity'].isin(extreme_values)]

# Data commands
commands = clean_df['command'].value_counts().reset_index()
commands.columns = ['command', 'counts']

# Upload data on intervals
lc = ListConventer(commands,'width_headers')
lc.conventer()
IntervalsUpload(gic.token, intervals_link(170), lc.datsheets)   


Данные успешно загружены в контейнер
<Response [200]>
